# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, youâ€™re going to use Keras to build a convolutional neural network in a few dozen lines of code.

Youâ€™ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Kerasâ€™s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the networkâ€™s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that youâ€™ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import matplotlib.image as mpimg
import cv2
from IPython.display import Image
%matplotlib inline


# TODO: Implement load the data here.
# Load pickled data
import pickle
from PIL import Image


# TODO: fill this in based on where you saved the training and testing data
def get_data(tr, te):
    #training_file = "train.p"
    #testing_file = "test.p"
    training_file = tr
    testing_file = te
    with open(training_file, mode='rb') as f:

        train = pickle.load(f)

    with open(testing_file, mode='rb') as f:

        test = pickle.load(f)

        X_train, y_train = train['features'], train['labels']

        X_test, y_test = test['features'], test['labels']
        
        return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = get_data("train.p","test.p")

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation


## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [ ]:
# TODO: Implement data normalization here.
def normalize_set(image_set, output_flat =True):
    output_set=[]   
    for i in range(0, image_set.shape[0]):
        r_norm=np.array([])
        r,g,b = cv2.split(image_set[i])        
        r_flat, g_flat, b_flat = r.flatten(), g.flatten(), b.flatten()

        r_norm = cv2.normalize(r_flat, r_flat, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        g_norm = cv2.normalize(g_flat,g_flat, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        b_norm = cv2.normalize(b_flat,b_flat, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)


        if(output_flat):
            norm_extra_bracket = cv2.merge((r_norm,  g_norm, b_norm ))
            #print(norm_extra_bracket.shape,"f")
            norm = norm_extra_bracket[:,0,:]
            #print(norm.shape,"f_no_extra_bracket")          
            
        else:
            r_norm32, g_norm32, b_norm32 =np.reshape(r_norm, (32,32)), np.reshape(g_norm, (32,32)),np.reshape(b_norm, (32,32))
            norm  = cv2.merge((r_norm32, g_norm32, b_norm32))
        output_set.append(norm)

    return np.array(output_set)  
 
    
    


print(type(X_train))
print(X_train.shape) 
print(type(X_test))
print(X_test.shape) 



#print(X_train[0])
#print(type(X_train), 'b')
#print(X_train.shape, 'b')
#print(X_train[0])


#X_train =normalize_set(X_train, True)
X_train =normalize_set(X_train, True)
X_test =normalize_set(X_test, True)
print(X_train.shape)
print(X_test.shape)
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [ ]:
# TODO: Build a two-layer feedforward neural network with Keras here.

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

In [ ]:
#batch_size = 128
batch_size = 128
nb_classes = 43
nb_epoch = 2

In [ ]:
#print(X_train.shape)
#print(y_train.shape)
#print(X_test.shape)
#print(y_test.shape)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
#below is to normalize that was done above.
#X_train /= 255
#X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
# convert class vectors to binary class matrices !!!! changes to capital Y
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
print(X_train.shape)

def flatten_set(image_set):
    output_set=[]   
    for i in range(0, image_set.shape[0]):

        flat = image_set[i].flatten()
        output_set.append(flat)

    return np.array(output_set) 
X_train=flatten_set(X_train)
X_test=flatten_set(X_test)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
#Build the model
model = Sequential()

model.add(Dense(128, input_shape=(3072, ), name='mac_and_cheese'))
#32*32*3=3072

model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(43, name= 'output'))
model.add(Activation('softmax'))

model.summary()

# STOP: Do not change the tests below. Your implementation should pass these tests.
#assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [ ]:
# TODO: Compile and train the model here.

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [ ]:
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)


In [ ]:
# TODO: Split some of the training data into a validation dataset.
# TODO: Compile and train the model to measure validation accuracy.
# Get randomized datasets for training and validation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample

X_train, X_val, Y_train, Y_val = train_test_split(
    
    X_train,
    
    Y_train,
    
    test_size=0.25,
    
    random_state=832289)


#print('Training features and labels randomized and split.')


In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
X_train = X_train.astype('float32')
X_val= X_val.astype('float32')




model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))
score = model.evaluate(X_val, Y_val, verbose=0)
print('Val score:', score[0])
print('Val accuracy:', score[1])

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
      
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

**Validation Accuracy**: (87%)


## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [1]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.

'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import matplotlib.image as mpimg
import cv2
from IPython.display import Image
%matplotlib inline


# TODO: Implement load the data here.
# Load pickled data
import pickle
from PIL import Image


# TODO: fill this in based on where you saved the training and testing data
def get_data(tr, te):
    #training_file = "train.p"
    #testing_file = "test.p"
    training_file = tr
    testing_file = te
    with open(training_file, mode='rb') as f:

        train = pickle.load(f)

    with open(testing_file, mode='rb') as f:

        test = pickle.load(f)

        X_train, y_train = train['features'], train['labels']

        X_test, y_test = test['features'], test['labels']
        
        return X_train, y_train, X_test, y_test

#X_train, y_train, X_test, y_test = get_data("train.p","test.p")


In [3]:
batch_size = 30
#nb_classes = 10
nb_classes = 43
nb_epoch = 12


# input image dimensions
img_rows, img_cols = 32, 32
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)


#added for final play ground
kernel_size_2 = (5, 5)

In [4]:
# the data, shuffled and split between train and test sets
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

#from above
X_train, y_train, X_test, y_test = get_data("train.p","test.p")


In [5]:
print(type(X_train))
print(X_train.shape) 
print(type(X_test))
print(X_test.shape) 


def normalize_set(image_set, output_flat =True):
    output_set=[]   
    for i in range(0, image_set.shape[0]):
        r_norm=np.array([])
        r,g,b = cv2.split(image_set[i])        
        r_flat, g_flat, b_flat = r.flatten(), g.flatten(), b.flatten()

        r_norm = cv2.normalize(r_flat, r_flat, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        g_norm = cv2.normalize(g_flat,g_flat, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        b_norm = cv2.normalize(b_flat,b_flat, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)


        if(output_flat):
            norm_extra_bracket = cv2.merge((r_norm,  g_norm, b_norm ))
            #print(norm_extra_bracket.shape,"f")
            norm = norm_extra_bracket[:,0,:]
            #print(norm.shape,"f_no_extra_bracket")          
            
        else:
            r_norm32, g_norm32, b_norm32 =np.reshape(r_norm, (32,32)), np.reshape(g_norm, (32,32)),np.reshape(b_norm, (32,32))
            norm  = cv2.merge((r_norm32, g_norm32, b_norm32))
        output_set.append(norm)

    return np.array(output_set)  

print(type(X_train))
print(X_train.shape) 
print(type(X_test))
print(X_test.shape) 

#X_train =normalize_set(X_train, True)
X_train =normalize_set(X_train, False)
X_test =normalize_set(X_test, False)
print(X_train.shape)
print(X_test.shape)

<class 'numpy.ndarray'>
(39209, 32, 32, 3)
<class 'numpy.ndarray'>
(12630, 32, 32, 3)
<class 'numpy.ndarray'>
(39209, 32, 32, 3)
<class 'numpy.ndarray'>
(12630, 32, 32, 3)
(39209, 32, 32, 3)
(12630, 32, 32, 3)


In [6]:
print(K.image_dim_ordering())


tf


In [7]:

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)
    

In [8]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
#max min normalier used above
#X_train /= 255
#X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (39209, 32, 32, 3)
39209 train samples
12630 test samples


In [9]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


In [10]:
# TODO: Split some of the training data into a validation dataset.
# TODO: Compile and train the model to measure validation accuracy.
# Get randomized datasets for training and validation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample

X_train, X_val, Y_train, Y_val = train_test_split(
    
    X_train,
    
    Y_train,
    
    test_size=0.3,
    
    random_state=832289)


#print('Training features and labels randomized and split.')


In [11]:

#conctruct
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


model.summary()

#compile and train
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_val, Y_val))




score = model.evaluate(X_val, Y_val, verbose=0)
print('Val score:', score[0])
print('Val accuracy:', score[1])

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
#assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]
#below is correct above incorrectly looks at the 'val_acc' after the first epoch. A bug report has been filed. 
#This may be fixed in your version.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 30, 30, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 28, 28, 32)    9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 28, 28, 32)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

**Validation Accuracy**: (98.6%)
**Test Accuracy**: (91.8%)

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [13]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.



#conctruct
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


model.summary()

#compile and train
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_val, Y_val))




score = model.evaluate(X_val, Y_val, verbose=0)
print('Val score:', score[0])
print('Val accuracy:', score[1])

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 30, 30, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 28, 28, 32)    9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 28, 28, 32)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

**Validation Accuracy**: (98.9%)
**Test Accuracy: (92.4%)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [11]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.

#conctruct
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


model.summary()

#compile and train
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_val, Y_val))




score = model.evaluate(X_val, Y_val, verbose=0)
print('Val score:', score[0])
print('Val accuracy:', score[1])

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 30, 30, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 28, 28, 32)    9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 28, 28, 32)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [ ]:
**Validation Accuracy**: (99.1%)
**Test Accuracy: (95.4%)

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [11]:
#already done above

#with open('./test.p', mode='rb') as f:
#    test = pickle.load(f)
#X_test = test['features']
#y_test = test['labels']
#X_test = X_test.astype('float32')
#X_test /= 255
#X_test -= 0.5
#Y_test = np_utils.to_categorical(y_test, 43)


#Construct model
#Construct right branch
from keras.layers import Merge


right_branch = Sequential()
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
right_branch.add(Activation('relu'))
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
right_branch.add(Activation('relu'))
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
right_branch.add(Activation('relu'))
right_branch.add(MaxPooling2D(pool_size=pool_size))
right_branch.add(Dropout(0.25))
right_branch.add(Flatten())


#Construct left branch
left_branch = Sequential()
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size_2[1],
                        border_mode='valid',
                        input_shape=input_shape))
left_branch.add(Activation('relu'))
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size_2[1]))
left_branch.add(Activation('relu'))
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size_2[1]))
left_branch.add(Activation('relu'))
left_branch.add(MaxPooling2D(pool_size=pool_size))
left_branch.add(Dropout(0.25))
left_branch.add(Flatten())



#Merge branches and construct final model
merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)

final_model.add(Dense(128))
final_model.add(Activation('relu'))
final_model.add(Dropout(0.5))
final_model.add(Dense(nb_classes))
final_model.add(Activation('softmax'))

final_model.summary()

#compile and train
final_model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

history = final_model.fit([X_train, X_train], Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=([X_val,X_val], Y_val))

score = final_model.evaluate([X_val,X_val], Y_val, verbose=0)
print('Val score:', score[0])
print('Val accuracy:', score[1])

score = final_model.evaluate([X_test, X_test], Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 30, 28, 32)    1472        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 30, 28, 32)    0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
convolution2d_5 (Convolution2D)  (None, 28, 24, 32)    15392       activation_4[0][0]               
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 28, 24, 32)    0           convolution2d_5[0][0]            
___________________________________________________________________________________________

**Validation Accuracy**: (99.3%)
**Test Accuracy: (96.0%)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [12]:
score = final_model.evaluate([X_test, X_test], Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]


Test score: 0.168438842645
Test accuracy: 0.959699129048


**Test Accuracy: (96.0%)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.